In [6]:
!pip install langchain docarray tiktoken --quiet

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


<a href="https://colab.research.google.com/github/qxr777/llm-application-code/blob/master/langchain_expression_language/rag-search-example.ipynb" target="_parent">Open In Colab</a>

In [ ]:
# 三选一 加载OPENAI_API_KEY

# 本地运行，包含.env文件
# from dotenv import load_dotenv
# load_dotenv()

# 本地运行，不包含.env文件
# import os
# os.environ['OPENAI_API_KEY'] = 'sk-123456789'

# google colab运行，需要配置secret key
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

![rag-search-example.png](../asserts/img/rag-search-example.png "rag-search-example")

In [8]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'